In [1]:
#taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
#and this Github repo: https://github.com/nsadawi/Download-Large-File-From-Google-Drive-Using-Python
import requests
import os
from tqdm import tqdm

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    with open(destination, "wb") as f:
        for chunk in tqdm(
            response.iter_content(1048576),
            unit="MiB", desc=f"Downloading {destination}"
        ):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
def download_multiple(files={}):
    """
    Downloads Gdrive files from IDs into specified paths
    Takes dictionary of {"filepath": "ID"}
    
    Note: 
    filepath must include filename, i.e. /directory/file.ext
    filename does not need to match filename on Gdrive
    """
    
    for fp, ID in files.items():
        dir_name = os.path.dirname(fp)
        
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        
        download_file_from_google_drive(ID, fp)

In [8]:
"""
#for testing
files={ "1558M/model.ckpt.meta": "10HlhfjbVNSuchYa2hmk9U1OlSRK6ZZxy",
        "774M/model.ckpt.meta": "1-omUh72kLcFCKecKTn6RMduJh51dgnC-",
        "out/shchedule.pdf": "1SZ76oFavmUeXeee1GElfpfwULzOxVUEH"
      }
"""

# Actual downloads
files={ "774M/model.ckpt.data-00000-of-00001": "1-iNTSsuloHKMzZA-ZDLSHN88RbWa3WrN",
        "1558M/model.ckpt.data-00000-of-00001": "107pyhj1vKojyoiFS0GJTyo0AWZNntUyo"
      }

download_multiple(files)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))